In [1]:
import requests
from bs4 import BeautifulSoup

base_url = "https://lescommis.com/recettes/page/"
all_links = []

# Supposons qu'il y ait 135 pages. Vous pouvez adapter au besoin
for page_num in range(1, 136):
    url = f"{base_url}{page_num}"
    print(f"Scraping {url}...")

    # Envoyer la requête HTTP
    response = requests.get(url)
    
    # Vérifier la validité de la réponse
    if response.status_code != 200:
        print(f"Page {page_num} indisponible, on arrête.")
        break  # on sort de la boucle si page inexistante/erreur

    # Parser le HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Sélectionner toutes les balises <a> menant aux recettes
    # (adapter le sélecteur si nécessaire)
    a_tags = soup.select("a[class*='ellipsis-3-lines']")  # par exemple
    # ou soup.select("a[data-v-5cc5e544]") si c'est un bon identifiant
    
    for a in a_tags:
        lien = a.get("href")
        if lien:
            # Construire l'URL absolue si besoin
            if lien.startswith("/"):
                lien = "https://lescommis.com" + lien
            all_links.append(lien)

# Visualiser tous les liens collectés
print(f"Nombre de liens collectés : {len(all_links)}")
for link in all_links:
    print(link)

Scraping https://lescommis.com/recettes/page/1...
Scraping https://lescommis.com/recettes/page/2...
Scraping https://lescommis.com/recettes/page/3...
Scraping https://lescommis.com/recettes/page/4...
Scraping https://lescommis.com/recettes/page/5...
Scraping https://lescommis.com/recettes/page/6...
Scraping https://lescommis.com/recettes/page/7...
Scraping https://lescommis.com/recettes/page/8...
Scraping https://lescommis.com/recettes/page/9...
Scraping https://lescommis.com/recettes/page/10...
Scraping https://lescommis.com/recettes/page/11...
Scraping https://lescommis.com/recettes/page/12...
Scraping https://lescommis.com/recettes/page/13...
Scraping https://lescommis.com/recettes/page/14...
Scraping https://lescommis.com/recettes/page/15...
Scraping https://lescommis.com/recettes/page/16...
Scraping https://lescommis.com/recettes/page/17...
Scraping https://lescommis.com/recettes/page/18...
Scraping https://lescommis.com/recettes/page/19...
Scraping https://lescommis.com/recettes/

In [3]:
len(all_links)

2677

Code pour scraper toutes les pages

In [4]:
# Importations nécessaires
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time
import csv
import pandas as pd
# Fonction pour nettoyer les chaînes de caractères
def clean_text(text):
    if text:
        text = text.replace("\n", " ").replace("\t", " ").replace("\xa0", " ").strip()
        text = text.replace("[", "").replace("]", "").replace("'", "").replace('"', "")
        return text
    return ""

# Fonction pour scraper les informations générales de la recette
def parse_recipe_bs(url):
    data = {}
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, "html.parser")

    # Titre
    titre_element = soup.find("h1", class_="header__name full-height")
    data["titre"] = clean_text(titre_element.get_text(separator=", ", strip=True)) if titre_element else "Titre introuvable"

    # Catch phrase
    catch_phrase_elem = soup.find("p", class_="header__catch-phrase")
    data["infos"] = clean_text(catch_phrase_elem.get_text(strip=True)) if catch_phrase_elem else ""

    # Ingrédients
    data["ingredients"] = []
    ingredient_list = soup.select_one("ul.ingredient__list")
    if ingredient_list:
        li_elements = ingredient_list.select("li.ingredient")
        for item in li_elements:
            name_elem = item.select_one("div.ingredient__name")
            qty_elem = item.select_one("div.ingredient__quantity")
            name = clean_text(name_elem.get_text(strip=True)) if name_elem else "Nom introuvable"
            qty = clean_text(qty_elem.get_text(strip=True)) if qty_elem else "Quantité introuvable"
            data["ingredients"].append((name, qty))

    # Instructions
    data["instructions"] = []
    bloc_etapes = soup.select_one("div.col-xs-12.col-md-8")
    if bloc_etapes:
        etapes = bloc_etapes.select("li.col-xs-12.col-sm-6[id]")
        for etape in etapes:
            titre_elem = etape.select_one("div.step__title")
            instructions_elem = etape.select_one("div.step__instructions")
            titre_step = clean_text(titre_elem.get_text(separator=" ", strip=True)) if titre_elem else "Titre introuvable"
            instr_step = clean_text(instructions_elem.get_text(separator=" ", strip=True)) if instructions_elem else "Instructions introuvables"
            data["instructions"].append((titre_step, instr_step))

    # Temps de préparation
    time_elem = soup.select_one("div.header__time-value.text-weight-bold.text-no-wrap")
    data["temps_preparation"] = clean_text(time_elem.get_text(strip=True)) if time_elem else ""

    # Infos régime
    data["infos_regime"] = []
    parent_div = soup.find("div", {"data-v-eb153f98": True, "class": "row q-col-gutter-sm"})
    if parent_div:
        hashtag_divs = parent_div.find_all("div", {"data-v-eb153f98": True})
        for div in hashtag_divs:
            text = clean_text(div.get_text(strip=True))
            if text.startswith("#"):
                text = text[1:]
            if text:
                data["infos_regime"].append(text)

    # Valeurs nutritionnelles (laisser vide ici, sera rempli par Selenium)
    data["valeurs_nutri_100g"] = []
    data["valeurs_nutri_portion"] = []
    return data

# Fonction pour scraper les données nutritionnelles avec Selenium
def parse_nutrition_selenium(url):
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    time.sleep(3)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    nutrition_data = []
    table = soup.select_one("table.q-table")
    if table:
        tbody = table.select_one("tbody.q-virtual-scroll__content")
        if tbody:
            rows = tbody.find_all("tr")
            for row in rows:
                cells = row.find_all("td")
                if len(cells) == 3:
                    nutriment = cells[0].get_text(strip=True)
                    val_100g = cells[1].get_text(strip=True).replace("\u202f", " ")
                    val_portion = cells[2].get_text(strip=True).replace("\u202f", " ")
                    nutrition_data.append((nutriment, val_100g, val_portion))
    return nutrition_data

# Organiser les données dans un dictionnaire final
def organiser_donnees_nutrition(url):
    data = parse_recipe_bs(url)
    nutrition_list = parse_nutrition_selenium(url)

    data['valeurs_nutri_100g'] = {}
    data['valeurs_nutri_portion'] = {}

    for nutriment, val_100g, val_portion in nutrition_list:
        data['valeurs_nutri_100g'][nutriment] = val_100g
        data['valeurs_nutri_portion'][nutriment] = val_portion

    return data


def export_to_csv(data, filename):
    """
    Exporte les données nettoyées dans un fichier CSV en utilisant Pandas DataFrame.
    """
    # Créer un dictionnaire structuré pour le DataFrame
    df_data = {
        "Titre": [clean_text(data["titre"])],
        "Infos": [clean_text(data["infos"])],
        "Ingrédients": ["; ".join([f"{ing} : {qty}" for ing, qty in data["ingredients"]])],
        "Instructions": [" | ".join([f"{step} : {instr}" for step, instr in data["instructions"]])],
        "Temps de préparation": [clean_text(data["temps_preparation"])],
        "Infos régime": [", ".join(data["infos_regime"])],
        "Valeurs pour 100g": ["; ".join([f"{nutriment} : {val}" for nutriment, val in data["valeurs_nutri_100g"].items()])],
        "Valeurs par portion": ["; ".join([f"{nutriment} : {val}" for nutriment, val in data["valeurs_nutri_portion"].items()])],
    }

    # Créer un DataFrame Pandas
    df = pd.DataFrame(df_data)

    # Exporter vers un fichier CSV
    df.to_csv(filename, sep=";", encoding="utf-8-sig", index=False)
    print(f"Les données ont été exportées dans le fichier {filename}.")



In [12]:
import os

def scrape_all_recipes(all_links, output_filename):
    """
    Boucle sur toutes les URLs dans all_links, scrape les données de chaque recette,
    et les exporte dans un fichier CSV avec Pandas.
    """
    # Vérifiez si le fichier CSV existe déjà
    file_exists = os.path.isfile(output_filename)

    # Ouvrir le fichier en mode ajout (append)
    with open(output_filename, mode='a', encoding="utf-8-sig", newline='') as file:
        # Définir les colonnes pour le CSV
        fieldnames = [
            "Titre", "Infos", "Ingrédients", "Instructions", 
            "Temps de préparation", "Infos régime", 
            "Valeurs pour 100g", "Valeurs par portion"
        ]
        writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=';')

        # Si le fichier n'existait pas, écrire l'en-tête
        if not file_exists:
            writer.writeheader()

        # Scraper chaque URL
        for url in all_links:
            print(f"Scraping : {url}")
            try:
                # Organiser les données de la recette
                data = organiser_donnees_nutrition(url)

                # Structurer les données dans un format compatible avec un DataFrame
                recipe_data = {
                    "Titre": clean_text(data["titre"]),
                    "Infos": clean_text(data["infos"]),
                    "Ingrédients": "; ".join([f"{ing} : {qty}" for ing, qty in data["ingredients"]]),
                    "Instructions": " | ".join([f"{step} : {instr}" for step, instr in data["instructions"]]),
                    "Temps de préparation": clean_text(data["temps_preparation"]),
                    "Infos régime": ", ".join(data["infos_regime"]),
                    "Valeurs pour 100g": "; ".join([f"{nutriment} : {val}" for nutriment, val in data["valeurs_nutri_100g"].items()]),
                    "Valeurs par portion": "; ".join([f"{nutriment} : {val}" for nutriment, val in data["valeurs_nutri_portion"].items()]),
                }

                # Écrire les données dans le fichier CSV
                writer.writerow(recipe_data)
                print(f"Données ajoutées pour {data['titre']}")

            except Exception as e:
                # Enregistrer une erreur dans les logs
                print(f"Erreur lors du scraping de {url} : {e}")


In [13]:
scrape_all_recipes(all_links, "recettes_rag.csv")

Scraping : https://lescommis.com/recettes/poisson-au-paprika-fume-legumes-rotis/


The chromedriver version (131.0.6778.204) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (132.0.6834.111); currently, chromedriver 132.0.6834.110 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


Données ajoutées pour Poisson au Paprika Fumé, Légumes Rôtis & Sauce Vierge
Scraping : https://lescommis.com/recettes/linguine-sauce-cremeuse-au-fromage-bio/


The chromedriver version (131.0.6778.204) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (132.0.6834.111); currently, chromedriver 132.0.6834.110 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


Données ajoutées pour Linguine Sauce Crémeuse, au Gorgonzola
Scraping : https://lescommis.com/recettes/veloute-de-panais-croustilles-de-pain-depicesbio/


...

Données ajoutées pour Burger, au Saint-Nectaire & Frites Maison
Scraping : https://lescommis.com/recettes/thermomix-brioche-super-bonne/


The chromedriver version (131.0.6778.204) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (132.0.6834.111); currently, chromedriver 132.0.6834.110 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


Données ajoutées pour Brioche Super Bonne
Scraping : https://lescommis.com/recettes/galette-des-rois-a-la-frangipane/


The chromedriver version (131.0.6778.204) detected in PATH at chromedriver.EXE might not be compatible with the detected chrome version (132.0.6834.111); currently, chromedriver 132.0.6834.110 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


Données ajoutées pour Galette des Rois, à la Frangipane


In [ ]:
import pandas as pd

def get_scraped_urls(csv_file):
    """
    Lit le fichier CSV et retourne une liste des titres déjà présents.
    """
    if os.path.exists(csv_file):
        df = pd.read_csv(csv_file, sep=";", encoding="utf-8-sig")
        return df["Titre"].tolist()
    return []

# Exclure les URLs déjà traitées
scraped_titles = get_scraped_urls("recettes_rag.csv")
all_links = [url for url in all_links if url not in scraped_titles]

scrape_all_recipes(all_links, "recettes_rag.csv")
